In [1]:
import datasets
import argparse
import os
import numpy as np
import sys
sys.path.append("../retrievers")
import texar as tx

from tqdm import tqdm
from totto_retriever import TottoRetriever
from transformers import T5Tokenizer, T5ForConditionalGeneration




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





















In [2]:
os.environ["HF_HOME"] = "/projects/ogma2/users/andrewsi/cache/huggingface/"
root = "/projects/ogma2/users/andrewsi/control-data2text"

In [3]:
model = T5ForConditionalGeneration.from_pretrained("t5-small", return_dict=True).to("cuda:0")

In [4]:
model.train()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Dro

In [6]:
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Dro

In [7]:
model.training

False

In [3]:
totto_dataset = f"{root}/prototype-retrieval/datasets/totto/totto_validation"
totto_val_json = f"{root}/transformers/examples/seq2seq/test_data/totto/val_with_headers_only.json"
retriever = TottoRetriever(totto_dataset)

In [4]:
retriever.dataset

Dataset({
    features: ['source', 'source_headers_only', 'source_headers_only_embed', 'target'],
    num_rows: 7700
})

In [6]:
TottoRetriever.mask_target(retriever.dataset[0]["source"], retriever.dataset[0]["target"])

'[MASK] was the [MASK] [MASK] of [MASK] from [MASK] .'

In [5]:
retriever.write_eval_set("/projects/ogma2/users/andrewsi/control-data2text/transformers/examples/seq2seq/test_data/totto_eval/val_headers_only.json", retrieval_embed="source_headers_only_embed", eval_k=5, gpu=0)

100%|██████████| 7700/7700 [01:19<00:00, 96.28it/s]
Wrote eval data to file: /projects/ogma2/users/andrewsi/control-data2text/transformers/examples/seq2seq/test_data/totto_eval/val_headers_only.json


In [5]:
val_data = retriever.dataset
val_data.add_faiss_index("source_headers_only_embed")

Dataset({
    features: ['source', 'source_headers_only', 'source_headers_only_embed', 'target'],
    num_rows: 7700
})

In [6]:
train_retriever = TottoRetriever("../datasets/totto/totto_train")

In [9]:
train_retriever.create_train_set("/projects/ogma2/users/andrewsi/control-data2text/transformers/examples/seq2seq/test_data/totto/train_k5.json", retrieval_k=5)

100%|██████████| 120761/120761 [04:09<00:00, 483.35it/s]
Wrote training data to file: /projects/ogma2/users/andrewsi/control-data2text/transformers/examples/seq2seq/test_data/totto/train_k5.json/train.json


In [7]:
idx = 1502

results = val_data.get_nearest_examples("source_headers_only_embed", np.array(val_data[idx]["source_headers_only_embed"], dtype=np.float32), k=30)

print(val_data[idx]["target"])
print("===================================")

for i, target in enumerate(results[1]["target"]):
        print("({}) {}".format(results[0][i], target))

Tejada made streak at 1,152 games.
(0.0) Tejada made streak at 1,152 games.
(30.03101348876953) Before Gehrig, Everett Scott made (1,307 consecutive games), whose streak ended in 1925, less than a month before Gehrig's began.
(45.333290100097656) Canada's Alice Falaiye twice (third and fourth attempt) had a winning jump with 6.43 metres.
(45.58599853515625) In 1989, President George H. W. Bush nominated John Tower to be Secretary of Defense.
(50.24248123168945) Mauer was selected by the Twins as the first overall pick of the 2001 draft.
(50.68273162841797) The 999 cc engine has a 71.1 mm (2.80 in) bore and a 62.9 mm (2.48 in) stroke.
(50.99052810668945) Heer received 275, 17 went to the Kriegsmarine, 117 to the Luftwaffe, and 28 to the Waffen-SS.
(52.408973693847656) Timo won Big Brother Germany 2010 with 75.2% of the votes against Marc.
(54.28458023071289) DNSSEC was added in Dnsmasq 2.69.
(54.964969635009766) The Redskins drafted Los Angeles Rams' seventh-round selection (241st overa